In [2]:
import os
import pandas as pd
import numpy as np
import re
from utils import load_eia_data

ImportError: cannot import name 'load_eia_data' from 'utils' (/Users/kamrantehranchi/Local_Documents/powerplants_usa/src/utils.py)

### Load EIA Data

In [ ]:
eia_data_operable, eia_storage, eia_loc, eia_plants_locs = load_eia_data(filter_region= "WECC",base_dir = '../..')

Goal is to have only ADS referenced generators in the network. Only need for EIA database is the locations

### Load ADS Generator List, Thermal General Info, and IOCurves.
Merge tables on various IDS

In [ ]:
ADS_PATH = os.path.join('../../data/WECC_ADS/downloads/2032/Public Data')
ads_thermal= pd.read_csv(ADS_PATH + '/Thermal_General_Info.csv',skiprows=1, )#encoding='unicode_escape')
ads_thermal = ads_thermal[['GeneratorName', ' Turbine Type', 'Model Type', 'MustRun',
       'MinimumDownTime(hr)', 'MinimumUpTime(hr)', 'MaxUpTime(hr)',
       'Generic Ramp Rate Name', 'RampUp Rate(MW/minute)',
       'RampDn Rate(MW/minute)', 'Startup Cost Fixed($)', 'StartFuel(MMBTu)', 'Startup Time',
       'VOM Cost']]

# Combine IOC data with the General Thermal Plant Data
ads_ioc= pd.read_csv(ADS_PATH + '/Thermal_IOCurve_Info.csv',skiprows=1, ).rename(columns={'Generator Name':'GeneratorName'})
ads_ioc = ads_ioc[[
       'GeneratorName', 'IOMaxCap(MW)','IOMinCap(MW)', 'MinInput(MMBTu)', 
       'IncCap2(MW)','IncHR2(MMBTu/MWh)','IncCap3(MW)', 'IncHR3(MMBTu/MWh)', 
       'IncCap4(MW)','IncHR4(MMBTu/MWh)','IncCap5(MW)', 'IncHR5(MMBTu/MWh)',
       'IncCap6(MW)','IncHR6(MMBTu/MWh)','IncCap7(MW)', 'IncHR7(MMBTu/MWh)']]

#Replacing heat rates with 0 with NaN
ads_ioc['IncHR2(MMBTu/MWh)'] = ads_ioc['IncHR2(MMBTu/MWh)'].replace(0, np.nan)

print('Size of thermal generator data: ', ads_thermal.shape)
print('Size of IOC generator data: ', ads_ioc.shape)
thermal_plant_data = pd.merge(ads_thermal, ads_ioc, on='GeneratorName', how='left')
thermal_plant_data

In [ ]:
#loading ads to match ads_name with generator key in order to link with ads thermal file
ads= pd.read_csv(ADS_PATH + '/GeneratorList.csv',skiprows=2, encoding='unicode_escape')
ads['Long Name'] = ads['Long Name'].astype(str)
ads.rename({'Name': 'ads_name', 'Long Name': 'ads_long_name',
             'SubType': 'subtype','Commission Date':'commission_date',
             'Retirement Date':'retirement_date','Area Name':'balancing_area'},
               axis=1, inplace=True)
ads.rename(str.lower, axis='columns', inplace=True)
ads['long id'] = ads['long id'].astype(str)
ads['tech_type'] = ads['fueltype'] + "_" + ads['technology']
ads_name_key_dict = dict(zip(ads['ads_name'], ads['generatorkey']))
print('Size of ads data: ', ads.shape)


# # Merge ads data with renewable plant data
cols = ['GeneratorKey', 'GeneratorName', 'Type',
       'DispatchShapeName', 'DispatchMultiplier', 'Capacity(MW)',
       ]
ads_hourly = pd.read_csv(ADS_PATH + '/HourlyResource_General.csv', skiprows=2,  encoding='unicode_escape')[cols]
ads_hourly.rename(columns={'Capacity(MW)': 'capacity_hourly_rsc'}, inplace=True)
# # Check if DispatchMultiplier is within 10 of Capacity(MW)
# tolerance = 100
# diff = abs(ads_hourly['DispatchMultiplier'] - ads_hourly['Capacity(MW)'])
# within_tolerance = (diff <= tolerance)
# ads_hourly[~within_tolerance]
ads = pd.merge(ads,ads_hourly.drop_duplicates(subset=['GeneratorName']), left_on='ads_name', right_on='GeneratorName', how='left')
print("Size of ads data: ", ads.shape)

In [ ]:
ads_thermal = pd.merge(ads, thermal_plant_data, left_on='ads_name', right_on='GeneratorName', how='left')

In [ ]:
eia_ads_mapping = pd.read_csv('../../repo_data/eia_mappings/eia_ads_generator_mapping_updated.csv')
cols = ['generatorkey', 'plant_id_eia', 'lat', 'lon']
ads_thermal_locs = pd.merge(ads_thermal, eia_ads_mapping[cols], on= 'generatorkey', how='inner')
print("Percent of Plant Locations Matched: ", round(100*ads_thermal_locs.shape[0]/ads_thermal.shape[0], 2), "%")
print("Percent of Capacity Matched: ", round(100*ads_thermal_locs['maxcap(mw)'].sum()/ads_thermal['maxcap(mw)'].sum(), 2), "%")
ads_thermal_locs

In [ ]:
#Assign locations to the Buses which we know the location of from other matches
bus_name_locs = ads_thermal_locs[['bus name', 'lat', 'lon']].groupby(['bus name']).mean().reset_index()
ads_unmatched = ads_thermal[~ads_thermal['generatorkey'].isin(ads_thermal_locs['generatorkey'])]
ads_unmatched_buses_known = ads_unmatched[ads_unmatched['bus name'].isin(bus_name_locs['bus name'])]
ads_unmatched_buses_known['lat'] = ads_unmatched_buses_known['bus name'].map(bus_name_locs.set_index('bus name')['lat'])
ads_unmatched_buses_known['lon'] = ads_unmatched_buses_known['bus name'].map(bus_name_locs.set_index('bus name')['lon'])
ads_unmatched_buses_known
ads_thermal_locs = pd.concat([ads_thermal_locs, ads_unmatched_buses_known])
print("Percent of plants located: ", round(100*len(ads_thermal_locs)/len(ads_thermal),2), "%")
print("Percent of Capacity Matched: ", round(100*ads_thermal_locs['maxcap(mw)'].sum()/ads_thermal['maxcap(mw)'].sum(), 2), "%")
ads_thermal_locs


In [ ]:
ads_non_usa

In [ ]:
# Concat the Non-US Plants which we know location of external bus
ads_unmatched = ads_thermal[~ads_thermal['generatorkey'].isin(ads_thermal_locs['generatorkey'])]
BA_known = ['AESO', 'BCHA', 'CFE']
ads_non_usa = ads_unmatched[ads_unmatched['balancing_area'].isin(BA_known)]

#Set Location of the external Buses
ads_non_usa['lat'] = ads_non_usa['balancing_area'].map({'AESO': 53.5461, 'BCHA': 49.2827, 'CFE': 31.85437})
ads_non_usa['lon'] = ads_non_usa['balancing_area'].map({'AESO': -113.4938, 'BCHA': -123.1207, 'CFE': -115.90479})

# Concat the Non-US Plants which we know location of external bus
ads_thermal_locs = pd.concat([ads_thermal_locs, ads_non_usa])
print("Precent of plants in known balancing areas: ", round(100*len(ads_non_usa)/len(ads_thermal),2), "%")
print("Percent of plants with Locs: ", round(100*len(ads_thermal_locs)/len(ads_thermal),2), "%")
print("Percent of plant Capacity: ", round(100*ads_thermal_locs['maxcap(mw)'].sum()/ads_thermal['maxcap(mw)'].sum(), 2), "%")

In [ ]:
ads_thermal_locs.to_csv('ads_thermal_locs.csv', index=False)

In [ ]:
ads_unmatched = ads_thermal[~ads_thermal['generatorkey'].isin(ads_thermal_locs['generatorkey'])]
ads_unmatched = ads_unmatched.sort_values(by='maxcap(mw)', ascending=False)
ads_unmatched.to_csv('ads_unmatched.csv', index=False)
ads_unmatched

In [ ]:
# from geopy.geocoders import Nominatim

# geolocator = Nominatim(user_agent="my_app")

# def get_lat_long(address):
#     location = geolocator.geocode(address)
#     if location:
#         return location.latitude, location.longitude
#     else:
#         return None


# ads_address = ads_unmatched[~ads_unmatched.county.isna()]
# addresses = ads_address.county + ", " + ads_address.state
# ads_address['lat'] = addresses.apply(lambda x: get_lat_long(x)[0] if get_lat_long(x) else None)
# ads_address['lon'] = addresses.apply(lambda x: get_lat_long(x)[1] if get_lat_long(x) else None)
# ads_address

In [ ]:
ads_unmatched

In [ ]:
import matplotlib.pyplot as plt

plt.hist(ads_unmatched['maxcap(mw)'],bins=50)
plt.xlabel('Capacity (MW)')
plt.ylabel('Frequency')
plt.title('Histogram of Power Plant Capacity')
plt.show()


### Final HotFixes and Exporting

In [ ]:
# Hot fix for mis-labeled wind plant 
ads_thermal_locs.loc[ads_thermal_locs.ads_name == 'Voyager Wind 1_3','fueltype'] = 'Wind'
ads_thermal_locs.loc[ads_thermal_locs.ads_name == 'Dracker Solar Unit 3 BESS','fueltype'] = 'battery'
ads_thermal_locs.loc[ads_thermal_locs.ads_name == 'Taos Mesa Storage','fueltype'] = 'battery'
ads_thermal_locs.fueltype.unique()

In [ ]:
ads_plants_locs = pd.concat([ads_thermal_locs, ads_unmatched])
ads_plants_locs.to_csv('../../repo_data/ads_plants_locs.csv', index=False)
ads_plants_locs

In [ ]:
import matplotlib.pyplot as plt

# First pie chart
subtype_counts = ads_thermal_locs['subtype'].value_counts()
subtype_counts.plot(kind='pie', autopct=lambda x: f'{int(x/100 * len(ads_unmatched))} ({x:.1f}%)' if x/100 * len(ads_unmatched) > 50 else '')
plt.title('ADS Matched by Subtype')
plt.show()


# Pie chart of maxcap(mw) of each subtype of plant
subtype_maxcap = ads_thermal_locs.groupby('subtype')['maxcap(mw)'].sum()
subtype_maxcap.plot(kind='pie', autopct='%1.1f%%')
plt.title('ADS Matched Capacity by Subtype')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# First pie chart
subtype_counts = ads_unmatched['subtype'].value_counts()
subtype_counts.plot(kind='pie', autopct=lambda x: f'{int(x/100 * len(ads_unmatched))} ({x:.1f}%)' if x/100 * len(ads_unmatched) > 50 else '')
plt.title('ADS Unmatched by Subtype')
plt.show()

# Second pie chart
ba_counts = ads_unmatched['balancing_area'].value_counts()
ba_counts.plot(kind='pie', autopct='%1.1f%%')
plt.title('ADS Unmatched by Balancing Area')
plt.show()


In [ ]:
#filter subtype dg-btm
filt = ['DG-BTM']
ads_unmatched[~ads_unmatched['subtype'].isin(filt)]